In [ ]:
!pip install pyterrier
!pip install python-terrier
!pip install sentence_transformers

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [ ]:
import pyterrier as pt
if not pt.started():
    pt.init()

<ipython-input-35-a9775fa13689>:2: DeprecationWarning: Call to deprecated function (or staticmethod) started. (use pt.java.started() instead) -- Deprecated since version 0.11.0.
  if not pt.started():


In [ ]:
df_q_explor= pd.read_csv("/content/drive/MyDrive/data_colab/inex_data/query_explor2005_agr.tsv", sep='\t')
df_q_explor['qid'] = df_q_explor['qid'].astype(str)
print(len(df_q_explor.index))

35


In [ ]:
print(len(df_q_explor.index))

35


In [ ]:
# Clean queries by removing line breaks and extra spaces
def clean_query(query):
    return ' '.join(query.replace('\n', ' ').split())
df_q_explor['query'] = df_q_explor['query'].apply(clean_query)

In [ ]:
df_q_explor

,qid,query,intent_fin
0,202,ontologies case study,exploratory
1,206,problems physical limits miniaturization micro...,exploratory
2,207,dom and sax,exploratory
3,208,artificial intelligence history,exploratory
4,210,multimedia document models content authoring,exploratory
5,213,gibbs sampler,exploratory
6,216,multimedia retrieval system architecture,exploratory
7,217,usercentered design of web sites,exploratory
8,219,learning object granularity,exploratory
9,222,eletronic commerce business strategies,exploratory


In [ ]:
index = pt.IndexFactory.of("/content/drive/MyDrive/data_colab/inex_index/sections_no_sw_2005")

In [ ]:



bm25 = pt.BatchRetrieve(index, wmodel="BM25", num_results=1500)


bm25_res = bm25.transform(df_q_explor)

rm3_pipe = bm25 >> pt.rewrite.RM3(index, fb_docs =5, fb_terms = 10) >> bm25

rm3_results = rm3_pipe.transform(df_q_explor)


# Retrieve the TF-IDF scores
tfidf = pt.BatchRetrieve(index, wmodel="TF_IDF", num_results=1500)
tfidf_res = tfidf.transform(df_q_explor)
tfidf_res


combined_results = pd.merge(rm3_results, tfidf_res[['qid', 'docid', 'score']], on=['qid', 'docid'], suffixes=('_bm25', '_tfidf'))

combined_results

<ipython-input-41-abb2a76cb550>:5: DeprecationWarning: Call to deprecated class BatchRetrieve. (use pt.terrier.Retriever() instead) -- Deprecated since version 0.11.0.
  bm25 = pt.BatchRetrieve(index, wmodel="BM25", num_results=1500)
<ipython-input-41-abb2a76cb550>:16: DeprecationWarning: Call to deprecated class BatchRetrieve. (use pt.terrier.Retriever() instead) -- Deprecated since version 0.11.0.
  tfidf = pt.BatchRetrieve(index, wmodel="TF_IDF", num_results=1500)


,qid,docid,docno,rank,score_bm25,query_0,query,score_tfidf
0,202,37946,ex/2001/x3066/article[1]/bdy[1]/sec[2],0,28.626876,ontologies case study,applypipeline:off case^0.200000018 photograph^...,11.487210
1,202,37951,ex/2001/x3066/article[1]/bdy[1]/sec[7],1,27.012216,ontologies case study,applypipeline:off case^0.200000018 photograph^...,11.176510
2,202,37947,ex/2001/x3066/article[1]/bdy[1]/sec[3],2,26.258647,ontologies case study,applypipeline:off case^0.200000018 photograph^...,11.173441
3,202,37948,ex/2001/x3066/article[1]/bdy[1]/sec[4],3,22.500993,ontologies case study,applypipeline:off case^0.200000018 photograph^...,6.954463
4,202,41197,ex/2002/x1054/article[1]/bdy[1]/sec[2],4,21.624783,ontologies case study,applypipeline:off case^0.200000018 photograph^...,8.551419
...,...,...,...,...,...,...,...,...
38820,286,68245,tc/1995/t0878/article[1]/bdy[1]/sec[5],1488,7.457913,interconnected networks,applypipeline:off fault^0.014997226 hot^0.0595...,4.857900
38821,286,33468,co/2003/r4018/article[1]/bdy[1]/sec[2],1490,7.455291,interconnected networks,applypipeline:off fault^0.014997226 hot^0.0595...,4.991982
38822,286,90668,ts/2000/e1066/article[1]/bdy[1]/sec[3],1496,7.444030,interconnected networks,applypipeline:off fault^0.014997226 hot^0.0595...,5.054864
38823,286,33817,co/2003/r4079/article[1]/bdy[1]/sec[2],1497,7.440414,interconnected networks,applypipeline:off fault^0.014997226 hot^0.0595...,4.917299


In [ ]:
l_q = combined_results['query_0'].to_list()
print(l_q[0])

ontologies case study


In [ ]:
import re

def clean_rm3_query(query):
    # Remove the weights (anything after ^)
    cleaned_query = re.sub(r'\^([^\s]+)', '', query)
    # Split the terms
    tokenized_query = cleaned_query.split()
    # Remove unnecessary tokens like 'applypipeline:off'
    tokenized_query = [term for term in tokenized_query if not term.startswith("applypipeline")]
    # Join the cleaned tokens back into a query string
    return ' '.join(tokenized_query)

# Apply the cleaning function to the 'query' column in the DataFrame
combined_results['query'] = combined_results['query'].apply(clean_rm3_query)


In [ ]:
combined_results

,qid,docid,docno,rank,score_bm25,query_0,query,score_tfidf
0,202,37946,ex/2001/x3066/article[1]/bdy[1]/sec[2],0,28.626876,ontologies case study,case photograph subject editor photo ontolog c...,11.487210
1,202,37951,ex/2001/x3066/article[1]/bdy[1]/sec[7],1,27.012216,ontologies case study,case photograph subject editor photo ontolog c...,11.176510
2,202,37947,ex/2001/x3066/article[1]/bdy[1]/sec[3],2,26.258647,ontologies case study,case photograph subject editor photo ontolog c...,11.173441
3,202,37948,ex/2001/x3066/article[1]/bdy[1]/sec[4],3,22.500993,ontologies case study,case photograph subject editor photo ontolog c...,6.954463
4,202,41197,ex/2002/x1054/article[1]/bdy[1]/sec[2],4,21.624783,ontologies case study,case photograph subject editor photo ontolog c...,8.551419
...,...,...,...,...,...,...,...,...
38820,286,68245,tc/1995/t0878/article[1]/bdy[1]/sec[5],1488,7.457913,interconnected networks,fault hot network symposium probabl chip analy...,4.857900
38821,286,33468,co/2003/r4018/article[1]/bdy[1]/sec[2],1490,7.455291,interconnected networks,fault hot network symposium probabl chip analy...,4.991982
38822,286,90668,ts/2000/e1066/article[1]/bdy[1]/sec[3],1496,7.444030,interconnected networks,fault hot network symposium probabl chip analy...,5.054864
38823,286,33817,co/2003/r4079/article[1]/bdy[1]/sec[2],1497,7.440414,interconnected networks,fault hot network symposium probabl chip analy...,4.917299


In [ ]:
import pandas as pd

df_th = pd.read_csv('/content/drive/MyDrive/data_colab/inex_data/thorough_no_empty_no_sw05_corr.tsv',sep="\t")
combined_results = pd.merge(combined_results, df_th[['docno', 'text']], on='docno', how='left')
combined_results

,qid,docid,docno,rank,score_bm25,query_0,query,score_tfidf,text
0,202,37946,ex/2001/x3066/article[1]/bdy[1]/sec[2],0,28.626876,ontologies case study,case photograph subject editor photo ontolog c...,11.487210,APPROACH Companies offering photographic image...
1,202,37951,ex/2001/x3066/article[1]/bdy[1]/sec[7],1,27.012216,ontologies case study,case photograph subject editor photo ontolog c...,11.176510,CONCLUSION study scratched surface regarding p...
2,202,37947,ex/2001/x3066/article[1]/bdy[1]/sec[3],2,26.258647,ontologies case study,case photograph subject editor photo ontolog c...,11.173441,DEVELOPING ONTOLOGIES define semantic annotati...
3,202,37948,ex/2001/x3066/article[1]/bdy[1]/sec[4],3,22.500993,ontologies case study,case photograph subject editor photo ontolog c...,6.954463,ANNOTATING PHOTOGRAPHS USING MULTIMEDIA INFORM...
4,202,41197,ex/2002/x1054/article[1]/bdy[1]/sec[2],4,21.624783,ontologies case study,case photograph subject editor photo ontolog c...,8.551419,ONTOLOGY LANGUAGES Several ontology languages ...
...,...,...,...,...,...,...,...,...,...
38820,286,68245,tc/1995/t0878/article[1]/bdy[1]/sec[5],1488,7.457913,interconnected networks,fault hot network symposium probabl chip analy...,4.857900,PERFORMANCE RESULTS evaluations discussed sect...
38821,286,33468,co/2003/r4018/article[1]/bdy[1]/sec[2],1490,7.455291,interconnected networks,fault hot network symposium probabl chip analy...,4.991982,FAST OPTICAL NETWORKING CHIPS Today's optical ...
38822,286,90668,ts/2000/e1066/article[1]/bdy[1]/sec[3],1496,7.444030,interconnected networks,fault hot network symposium probabl chip analy...,5.054864,Motivating Application application prompted de...
38823,286,33817,co/2003/r4079/article[1]/bdy[1]/sec[2],1497,7.440414,interconnected networks,fault hot network symposium probabl chip analy...,4.917299,CALL ARTICLES COMPUTER Mobile computing device...


1.INTENT AGNOSTIC FEATURES

In [ ]:
# Calculate the sum, min, max, mean, and variance of the 'score_tfidf' for each query (grouped by 'qid')
tfidf_stats = combined_results.groupby('qid')['score_tfidf'].agg(
    sum_tfidf='sum',
    min_tfidf='min',
    max_tfidf='max',
    mean_tfidf='mean',
    var_tfidf='var'
).reset_index()

# Display the calculated statistics
tfidf_stats

,qid,sum_tfidf,min_tfidf,max_tfidf,mean_tfidf,var_tfidf
0,202,6988.958534,4.229216,11.487210,6.136048,1.985902
1,206,6358.348840,5.375904,14.849340,7.112247,2.181690
2,207,1761.547647,1.358020,20.033968,6.827704,12.136526
3,208,9394.720031,4.762579,13.784484,7.085008,2.212134
4,210,11805.907434,7.350564,17.617513,9.528577,2.964893
5,213,1905.102390,1.552513,19.625483,6.828324,13.823184
6,216,8991.057706,6.272205,13.505790,8.196042,1.998934
7,217,9749.176567,6.604448,12.991104,7.749743,0.612076
8,219,6527.659426,4.464417,10.394913,5.564927,0.696147
9,222,9418.150318,5.024086,13.209488,7.145789,2.437046


In [ ]:
combined_results = pd.merge(combined_results, tfidf_stats, on='qid')

In [ ]:
# Download stopwords
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
combined_results['query_0'] = combined_results['query_0'].astype(str)
combined_results['text'] = combined_results['text'].astype(str)

In [ ]:
stop_words = set(stopwords.words('english'))

# Preprocess the text (tokenization, stopword removal)
def preprocess(text):
    tokens = word_tokenize(text.lower())
    tokens = [word for word in tokens if word.isalpha() and word not in stop_words]
    return tokens

# Apply preprocessing to both queries and documents
combined_results['query_tokens'] = combined_results['query_0'].apply(preprocess)
combined_results['document_tokens'] = combined_results['text'].apply(preprocess)

# Combine queries and documents into one list for further processing
combined_text = pd.concat([combined_results['query_tokens'].apply(' '.join), combined_results['document_tokens'].apply(' '.join)], axis=0)


In [ ]:
combined_results['document_length'] = combined_results['text'].str.split().str.len()

In [ ]:
def all_words_in_text(query, text):
    query_words = set(query.lower().split())  # Tokenizing query and converting to lowercase
    text_words = set(text.lower().split())    # Tokenizing text and converting to lowercase
    return 1 if query_words.issubset(text_words) else 0  # Return 1 if all query words are found, else 0

combined_results['query_in_text'] = combined_results.apply(lambda row: all_words_in_text(row['query_0'], row['text']), axis=1)

2.INTENT-AWARE FEATURES

In [ ]:
# TFIDF Cosine Similarity
# Convert tokenized text back to string for TF-IDF vectorizer
combined_results['query_clean'] = combined_results['query_tokens'].apply(' '.join)
combined_results['document_clean'] = combined_results['document_tokens'].apply(' '.join)

# Combine cleaned queries and documents
combined_clean_text = pd.concat([combined_results['query_clean'], combined_results['document_clean']], axis=0)

# Apply TFIDF Vectorizer
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(combined_clean_text)

# Split TFIDF vectors for queries and documents
query_tfidf = tfidf_matrix[:len(combined_results['query_0'])]  # TFIDF vectors for queries
doc_tfidf = tfidf_matrix[len(combined_results['query_0']):]    # TFIDF vectors for documents

# Compute cosine similarity between each query and its corresponding document
combined_results['tfidf_cosine_similarity'] = cosine_similarity(query_tfidf, doc_tfidf).diagonal()


In [ ]:
# Load the pretrained Sentence-BERT model
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

# Compute BERT embeddings for queries and documents
combined_results['query_embedding'] = combined_results['query_0'].apply(lambda x: model.encode(x, convert_to_tensor=False))
combined_results['document_embedding'] = combined_results['text'].apply(lambda x: model.encode(x, convert_to_tensor=False))

# Compute cosine similarity between each query and document using BERT embeddings
combined_results['bert_cosine_similarity'] = combined_results.apply(lambda row: cosine_similarity([row['query_embedding']], [row['document_embedding']])[0][0], axis=1)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
combined_results

,qid,docid,docno,rank,score_bm25,query_0,query,score_tfidf,text,sum_tfidf,...,query_tokens,document_tokens,document_length,query_in_text,query_clean,document_clean,tfidf_cosine_similarity,query_embedding,document_embedding,bert_cosine_similarity
0,202,37946,ex/2001/x3066/article[1]/bdy[1]/sec[2],0,28.626876,ontologies case study,case photograph subject editor photo ontolog c...,11.487210,APPROACH Companies offering photographic image...,6988.958534,...,"[ontologies, case, study]","[approach, companies, offering, photographic, ...",159,0,ontologies case study,approach companies offering photographic image...,0.213656,"[-0.24046423, 0.07049384, -0.29815373, -0.3256...","[-0.30226064, 0.16808213, -0.025826601, -0.048...",0.572751
1,202,37951,ex/2001/x3066/article[1]/bdy[1]/sec[7],1,27.012216,ontologies case study,case photograph subject editor photo ontolog c...,11.176510,CONCLUSION study scratched surface regarding p...,6988.958534,...,"[ontologies, case, study]","[conclusion, study, scratched, surface, regard...",522,1,ontologies case study,conclusion study scratched surface regarding p...,0.118830,"[-0.24046423, 0.07049384, -0.29815373, -0.3256...","[-0.19483483, 0.103270695, 0.032962322, 0.1470...",0.543533
2,202,37947,ex/2001/x3066/article[1]/bdy[1]/sec[3],2,26.258647,ontologies case study,case photograph subject editor photo ontolog c...,11.173441,DEVELOPING ONTOLOGIES define semantic annotati...,6988.958534,...,"[ontologies, case, study]","[developing, ontologies, define, semantic, ann...",937,0,ontologies case study,developing ontologies define semantic annotati...,0.062881,"[-0.24046423, 0.07049384, -0.29815373, -0.3256...","[-0.14975718, 0.15120077, 0.13548763, 0.036045...",0.574287
3,202,37948,ex/2001/x3066/article[1]/bdy[1]/sec[4],3,22.500993,ontologies case study,case photograph subject editor photo ontolog c...,6.954463,ANNOTATING PHOTOGRAPHS USING MULTIMEDIA INFORM...,6988.958534,...,"[ontologies, case, study]","[annotating, photographs, using, multimedia, i...",298,0,ontologies case study,annotating photographs using multimedia inform...,0.000000,"[-0.24046423, 0.07049384, -0.29815373, -0.3256...","[-0.3374124, -0.014775455, -0.28002357, -0.256...",0.280342
4,202,41197,ex/2002/x1054/article[1]/bdy[1]/sec[2],4,21.624783,ontologies case study,case photograph subject editor photo ontolog c...,8.551419,ONTOLOGY LANGUAGES Several ontology languages ...,6988.958534,...,"[ontologies, case, study]","[ontology, languages, several, ontology, langu...",533,0,ontologies case study,ontology languages several ontology languages ...,0.059484,"[-0.24046423, 0.07049384, -0.29815373, -0.3256...","[-0.57515246, -0.14130484, -0.06348605, -0.099...",0.628480
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38820,286,68245,tc/1995/t0878/article[1]/bdy[1]/sec[5],1488,7.457913,interconnected networks,fault hot network symposium probabl chip analy...,4.857900,PERFORMANCE RESULTS evaluations discussed sect...,7418.357917,...,"[interconnected, networks]","[performance, results, evaluations, discussed,...",1672,0,interconnected networks,performance results evaluations discussed sect...,0.000000,"[-0.5767809, -0.30667368, 0.12936193, -0.40115...","[-0.18369228, -0.11817601, 0.00793449, -0.3195...",0.237010
38821,286,33468,co/2003/r4018/article[1]/bdy[1]/sec[2],1490,7.455291,interconnected networks,fault hot network symposium probabl chip analy...,4.991982,FAST OPTICAL NETWORKING CHIPS Today's optical ...,7418.357917,...,"[interconnected, networks]","[fast, optical, networking, chips, today, opti...",476,0,interconnected networks,fast optical networking chips today optical ne...,0.036360,"[-0.5767809, -0.30667368, 0.12936193, -0.40115...","[-0.11976287, 0.10906428, 0.08707214, -0.22543...",0.338498
38822,286,90668,ts/2000/e1066/article[1]/bdy[1]/sec[3],1496,7.444030,interconnected networks,fault hot network symposium probabl chip analy...,5.054864,Motivating Application application prompted de...,7418.357

In [ ]:
combined_results = combined_results.drop(columns=['query_embedding', 'document_embedding', 'query_tokens', 'document_tokens', 'query_clean', 'document_clean'])

In [ ]:
combined_results

,qid,docid,docno,rank,score_bm25,query_0,query,score_tfidf,text,sum_tfidf,min_tfidf,max_tfidf,mean_tfidf,var_tfidf,document_length,query_in_text,tfidf_cosine_similarity,bert_cosine_similarity
0,202,37946,ex/2001/x3066/article[1]/bdy[1]/sec[2],0,28.626876,ontologies case study,case photograph subject editor photo ontolog c...,11.487210,APPROACH Companies offering photographic image...,6988.958534,4.229216,11.487210,6.136048,1.985902,159,0,0.213656,0.572751
1,202,37951,ex/2001/x3066/article[1]/bdy[1]/sec[7],1,27.012216,ontologies case study,case photograph subject editor photo ontolog c...,11.176510,CONCLUSION study scratched surface regarding p...,6988.958534,4.229216,11.487210,6.136048,1.985902,522,1,0.118830,0.543533
2,202,37947,ex/2001/x3066/article[1]/bdy[1]/sec[3],2,26.258647,ontologies case study,case photograph subject editor photo ontolog c...,11.173441,DEVELOPING ONTOLOGIES define semantic annotati...,6988.958534,4.229216,11.487210,6.136048,1.985902,937,0,0.062881,0.574287
3,202,37948,ex/2001/x3066/article[1]/bdy[1]/sec[4],3,22.500993,ontologies case study,case photograph subject editor photo ontolog c...,6.954463,ANNOTATING PHOTOGRAPHS USING MULTIMEDIA INFORM...,6988.958534,4.229216,11.487210,6.136048,1.985902,298,0,0.000000,0.280342
4,202,41197,ex/2002/x1054/article[1]/bdy[1]/sec[2],4,21.624783,ontologies case study,case photograph subject editor photo ontolog c...,8.551419,ONTOLOGY LANGUAGES Several ontology languages ...,6988.958534,4.229216,11.487210,6.136048,1.985902,533,0,0.059484,0.628480
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38820,286,68245,tc/1995/t0878/article[1]/bdy[1]/sec[5],1488,7.457913,interconnected networks,fault hot network symposium probabl chip analy...,4.857900,PERFORMANCE RESULTS evaluations discussed sect...,7418.357917,4.714500,7.874099,5.958520,0.522739,1672,0,0.000000,0.237010
38821,286,33468,co/2003/r4018/article[1]/bdy[1]/sec[2],1490,7.455291,interconnected networks,fault hot network symposium probabl chip analy...,4.991982,FAST OPTICAL NETWORKING CHIPS Today's optical ...,7418.357917,4.714500,7.874099,5.958520,0.522739,476,0,0.036360,0.338498
38822,286,90668,ts/2000/e1066/article[1]/bdy[1]/sec[3],1496,7.444030,interconnected networks,fault hot network symposium probabl chip analy...,5.054864,Motivating Application application prompted de...,7418.357917,4.714500,7.874099,5.958520,0.522739,155,0,0.045231,0.139314
38823,286,33817,co/2003/r4079/article[1]/bdy[1]/sec[2],1497,7.440414,interconnected networks,fault hot network symposium probabl chip analy...,4.917299,CALL ARTICLES COMPUTER Mobile computing device...,7418.357917,4.714500,7.874099,5.958520,0.522739,449,0,0.123352,0.267484


In [ ]:
combined_results.to_csv('/content/drive/MyDrive/data_colab/inex_data/res_with_features_explor_2005_features_sections_RM3.tsv', sep='\t', index=False)